In [7]:
from matplotlib import pyplot as plt
import numpy as np
import json
import math
import csv
import os

In [8]:
def denormalization(arr, angle, translation):

    theta = (angle)/180*math.pi
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    
    #rotate
    arr = np.array(R.dot(arr.transpose())).transpose()
    #translate
    arr += translation
#     arr[...,0] += (AGENT[19,0] - AGENT[0,0])
#     arr[...,1] += (AGENT[19,1] - AGENT[0,1])
    return arr

In [9]:

def prediction(root_dir, file):
    json_dict = {}

    with open(root_dir + file, 'r') as json_data:
        json_dict = json.load(json_data)

    preds = np.array(json_dict['preds'][0])[...,:2]
    preds = np.array([denormalization(p, -json_dict['rotation'], -np.array(json_dict['translation'])) for p in preds])
    seq_id = int(json_dict['csv_file'].split('.')[0])
    return preds, seq_id
#     for p in preds:
#         plt.plot(p[...,0],p[...,1])
#         plt.scatter(p[...,0][-1],p[...,1][-1])


#     plt.plot()
#     plt.show()


In [16]:
from tqdm.notebook import tqdm
root_dir = "results/epoch116/"
file_list = os.listdir(root_dir)
output_all = {}
for file in tqdm(file_list):
    p, seq_id = prediction(root_dir, file)
    output_all[seq_id] = p 

In [17]:
from argoverse.evaluation.competition_util import generate_forecasting_h5

output_path = 'competition_files/competition116'

generate_forecasting_h5(output_all, output_path) #this might take awhile

78143/78143